# Obtain Evaluation on MIMIC dataset for OASIS and SAPS II

In [1]:
import mimic_pipeline.utils as utils
user = input('Enter your username: ')
password = input('Enter your password: ')
loader = utils.DataBaseLoader(user=user, password=password)
oasis_df = loader['oasis']
sapsii_df = loader['sapsii']
oasis_df.head()

,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,...,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,55973,152234,200001,adult,0,0,43,0.333566,61.071279,6,...,32.0,6.0,36.388889,2.0,250.0,10.0,0,0,0,6
1,27513,163557,200003,adult,0,0,35,0.152892,48.296271,3,...,39.0,6.0,36.388889,2.0,3652.0,0.0,1,9,0,6
2,10950,189514,200006,adult,0,0,32,0.109623,54.072308,6,...,27.0,1.0,36.166666,2.0,1955.0,1.0,1,9,0,6
3,20707,129310,200007,adult,0,0,26,0.054187,43.345013,3,...,29.0,1.0,36.388889,2.0,1295.0,5.0,0,0,0,6
4,29904,129607,200009,adult,0,0,25,0.048012,47.555978,3,...,17.5,0.0,34.599998,4.0,1570.0,1.0,1,9,1,0


In [2]:
sapsii_df.head()

,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,pao2fio2_score,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,55973,152234,200001,44,0.326364,12,4.0,5.0,0.0,NaN,11.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6
1,27513,163557,200003,30,0.106398,7,4.0,5.0,0.0,NaN,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0,8
2,10950,189514,200006,20,0.037205,7,2.0,5.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
3,20707,129310,200007,18,0.029295,7,0.0,5.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
4,29904,129607,200009,21,0.041754,7,0.0,5.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,NaN,0.0,0,0


In [3]:
import pandas as pd
train = pd.read_csv("data/TRAIN-union-features-excluded-cmo.csv")
test = pd.read_csv("data/TEST-union-features-excluded-cmo.csv")
all = pd.concat([train, test], axis=0)
print(all.shape)
all.head()

(29265, 50)


,preiculos,age,gcs_min,mechvent,urineoutput,heartrate_min,heartrate_max,meanbp_min,meanbp_max,resprate_min,...,admissiontype,pao2_max,pao2_min,paco2_max,paco2_min,ph_min,ph_max,aado2_min,aado2_max,hospital_expire_flag
0,4777.0,43.0,15.0,0,3980.0,68.0,118.0,77.000000,103.0,14.0,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,1.0,50.0,15.0,0,1050.0,77.0,120.0,56.000000,119.0,9.0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,238.0,38.0,15.0,1,2000.0,101.0,116.0,56.666698,91.0,8.0,...,0,458.0,139.0,52.0,34.0,7.21,7.47,284.0,284.0,-1
3,286.0,80.0,15.0,0,1145.0,46.0,76.0,51.000000,73.0,12.0,...,0,168.0,80.0,41.0,27.0,7.36,7.42,NaN,NaN,-1
4,1.0,75.0,13.0,0,3040.0,67.0,92.0,69.000000,104.0,12.0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [4]:
from sklearn.metrics import auc, precision_recall_curve, roc_curve, brier_score_loss
from sklearn.model_selection import StratifiedKFold
from mimic_pipeline.metric import get_calibration_curve
import numpy as np
import joblib

utils.seed_everything()
oasis_auroc, sapsii_auroc = [], []
oasis_auprc, sapsii_auprc = [], []
oasis_hl, sapsii_hl = [], []
oasis_brier, sapsii_brier = [], []
oasis_smr, sapsii_smr = [], []
for fold in range(1,6):
    test_df = pd.read_csv(f"data/k-fold-no-cmo/TEST-union-features-fold{fold}.csv")
    copy = test_df[['subject_id', 'hadm_id', 'icustay_id']].copy()
    test_df = test_df.merge(oasis_df[['subject_id', 'icustay_id', 'hadm_id', 'oasis_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
    test_df = test_df.merge(sapsii_df[['subject_id', 'icustay_id', 'hadm_id', 'sapsii_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
    pd.testing.assert_frame_equal(test_df[['subject_id', 'hadm_id', 'icustay_id']], copy)

    fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
    oasis_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['oasis_prob']) )
    oasis_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['oasis_prob']))
    oasis_auroc.append(auc(fpr, tpr))
    oasis_auprc.append(auc(recall, precision))
    oasis_hl.append(h)
    
    fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
    precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
    prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'], strategy='quantile')
    sapsii_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['sapsii_prob']))
    sapsii_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['sapsii_prob']))
    sapsii_auroc.append(auc(fpr, tpr))
    sapsii_auprc.append(auc(recall, precision))
    sapsii_hl.append(h)

oasis_stats = {'auroc': oasis_auroc, 'auprc': oasis_auprc, 'c-stat': oasis_hl, 'smr': oasis_smr, 'brier': oasis_brier}
sapsii_stats = {'auroc': sapsii_auroc, 'auprc': sapsii_auprc, 'c-stat': sapsii_hl, 'smr': sapsii_smr, 'brier': sapsii_brier}
joblib.dump(oasis_stats, "results/oasis_no_cmo_mimic-stats")
joblib.dump(sapsii_stats, "results/sapsii_no_cmo_mimic-stats")
    
print(f"OASIS AUROC: {np.mean(oasis_auroc):.3f} $\pm$ {np.std(oasis_auroc):.3f}")
print(f"OASIS AUPRC: {np.mean(oasis_auprc):.3f} $\pm$ {np.std(oasis_auprc):.3f}")
print(f"OASIS HL C-stat: {np.mean(oasis_hl):.3f} $\pm$ {np.std(oasis_hl):.3f}")
print(f"OASIS SMR: {np.mean(oasis_smr):.3f} $\pm$ {np.std(oasis_smr):.3f}")
print(f"OASIS Brier: {np.mean(oasis_brier):.3f} $\pm$ {np.std(oasis_brier):.3f}\n")
print(f"SAPSII AUROC: {np.mean(sapsii_auroc):.3f} $\pm$ {np.std(sapsii_auroc):.3f}")
print(f"SAPSII AUPRC: {np.mean(sapsii_auprc):.3f} $\pm$ {np.std(sapsii_auprc):.3f}")
print(f"SAPSII HL C-stat: {np.mean(sapsii_hl):.3f} $\pm$ {np.std(sapsii_hl):.3f}")
print(f"SAPSII SMR: {np.mean(sapsii_smr):.3f} $\pm$ {np.std(sapsii_smr):.3f}")
print(f"SAPSII Brier: {np.mean(sapsii_brier):.3f} $\pm$ {np.std(sapsii_brier):.4f}")

OASIS AUROC: 0.761 $\pm$ 0.006
OASIS AUPRC: 0.239 $\pm$ 0.016
OASIS HL C-stat: 296.694 $\pm$ 7.950
OASIS SMR: 0.523 $\pm$ 0.002
OASIS Brier: 0.073 $\pm$ 0.001

SAPSII AUROC: 0.788 $\pm$ 0.008
SAPSII AUPRC: 0.270 $\pm$ 0.008
SAPSII HL C-stat: 925.273 $\pm$ 12.365
SAPSII SMR: 0.370 $\pm$ 0.004
SAPSII Brier: 0.094 $\pm$ 0.0005


### Disease Specific

In [ ]:
sofa_df = loader['sofa_revised']
sofa_df.head()

In [ ]:
sofa_df['sofa_emp_prob'].unique()

In [ ]:
import joblib
utils.seed_everything()

def evaluate_baseline_disease(disease):
    print(f"Evaluating {disease}...")
    oasis_auroc, sapsii_auroc, sofa_auroc = [], [], []
    oasis_auprc, sapsii_auprc, sofa_auprc = [], [], []
    oasis_hl, sapsii_hl, sofa_hl = [], [], []
    oasis_brier, sapsii_brier, sofa_brier = [], [], []
    oasis_smr, sapsii_smr, sofa_smr = [], [], []
    for fold in range(1,6):
        test_df = pd.read_csv(f"data/mimic-disease/k-fold-id/TEST-{disease}-fold{fold}.csv")
        copy = test_df[['subject_id', 'hadm_id', 'icustay_id']].copy()
        test_df = test_df.merge(oasis_df[['subject_id', 'icustay_id', 'hadm_id', 'oasis_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        test_df = test_df.merge(sapsii_df[['subject_id', 'icustay_id', 'hadm_id', 'sapsii_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        test_df = test_df.merge(sofa_df[['subject_id', 'icustay_id', 'hadm_id', 'sofa_emp_prob']], on=['subject_id', 'icustay_id', 'hadm_id'], how='inner')
        pd.testing.assert_frame_equal(test_df[['subject_id', 'hadm_id', 'icustay_id']], copy)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['oasis_prob'])
        oasis_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['oasis_prob']) )
        oasis_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['oasis_prob']))
        oasis_auroc.append(auc(fpr, tpr))
        oasis_auprc.append(auc(recall, precision))
        oasis_hl.append(h)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'])
        prob_true, prob_pred, h, p = get_calibration_curve(test_df['hospital_expire_flag'], test_df['sapsii_prob'], strategy='quantile')
        sapsii_brier.append(brier_score_loss(test_df['hospital_expire_flag'], test_df['sapsii_prob']))
        sapsii_smr.append(np.sum(test_df['hospital_expire_flag'].replace({-1: 0})) / np.sum(test_df['sapsii_prob']))
        sapsii_auroc.append(auc(fpr, tpr))
        sapsii_auprc.append(auc(recall, precision))
        sapsii_hl.append(h)
        
        fpr, tpr, _ = roc_curve(test_df['hospital_expire_flag'], test_df['sofa_emp_prob'])
        precision, recall, _ = precision_recall_curve(test_df['hospital_expire_flag'], test_df['sofa_emp_prob'])
        sofa_auroc.append(auc(fpr, tpr))
        sofa_auprc.append(auc(recall, precision))
    
    sofa_stats = {'auroc': sofa_auroc, 'auprc': sofa_auprc}
    sapsii_stats = {'auroc': sapsii_auroc, "auprc": sapsii_auprc}
    oasis_stats = {'auroc': oasis_auroc, "auprc": oasis_auprc}
    
    joblib.dump(sofa_stats, f"results/disease/{disease}/sofa_stats")
    joblib.dump(sapsii_stats, f"results/disease/{disease}/sapsii_stats")
    joblib.dump(oasis_stats, f"results/disease/{disease}/oasis_stats")
        
    print(f"OASIS AUROC: {np.mean(oasis_auroc):.3f} +/- {np.std(oasis_auroc):.3f}")
    print(f"OASIS AUPRC: {np.mean(oasis_auprc):.3f} +/- {np.std(oasis_auprc):.3f}")
    print(f"OASIS HL C-stat: {np.mean(oasis_hl):.3f} +/- {np.std(oasis_hl):.3f}")
    print(f"OASIS SMR: {np.mean(oasis_smr):.3f} +/- {np.std(oasis_smr):.3f}")
    print(f"OASIS Brier: {np.mean(oasis_brier):.3f} +/- {np.std(oasis_brier):.3f}\n")
    
    print(f"SAPSII AUROC: {np.mean(sapsii_auroc):.3f} +/- {np.std(sapsii_auroc):.3f}")
    print(f"SAPSII AUPRC: {np.mean(sapsii_auprc):.3f} +/- {np.std(sapsii_auprc):.3f}")
    print(f"SAPSII HL C-stat: {np.mean(sapsii_hl):.3f} +/- {np.std(sapsii_hl):.3f}")
    print(f"SAPSII SMR: {np.mean(sapsii_smr):.3f} +/- {np.std(sapsii_smr):.3f}")
    print(f"SAPSII Brier: {np.mean(sapsii_brier):.3f} +/- {np.std(sapsii_brier):.4f}\n")
    
    print(f"SOFA AUROC: {np.mean(sofa_auroc):.3f} +/- {np.std(sofa_auroc):.3f}")
    print(f"SOFA AUPRC: {np.mean(sofa_auprc):.3f} +/- {np.std(sofa_auprc):.3f}")

evaluate_baseline_disease('sepsis')

In [ ]:
evaluate_baseline_disease('akf')

In [ ]:
evaluate_baseline_disease('ami')

In [ ]:
evaluate_baseline_disease('heart_failure')